In [0]:
!nvidia-smi

!pip install pytorch-nlp
!pip install pytorch-transformers



Sun Aug  4 08:48:23 2019       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 418.67       Driver Version: 410.79       CUDA Version: 10.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   29C    P8    27W / 149W |      0MiB / 11441MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

In [0]:
#Get Test Data from Dropbox
!git clone https://github.com/thatbrguy/Dropbox-Uploader.git
%cd Dropbox-Uploader
!chmod +x dropbox_uploader.sh
!bash dropbox_uploader.sh
!echo "CHTZbyeOjyYAAAAAAAAAfZTfC_d3kTmHXSqg7-941o9Nolj-FEAJbjyUKutEtCSu" > token.txt
!bash dropbox_uploader.sh

!bash dropbox_uploader.sh download xlnet_imdb_test.zip
!unzip xlnet_imdb_test.zip
!rm xlnet_imdb_test.zip

!bash dropbox_uploader.sh download xlnet_imdb_train.zip
!unzip xlnet_imdb_train.zip
!rm xlnet_imdb_train.zip

In [0]:


import numpy as np
import torch.nn as nn

from torchnlp.datasets import smt_dataset
from torchnlp.datasets import imdb_dataset
#from torchnlp.datasets import trec_dataset

import torch
from pytorch_transformers import XLNetForSequenceClassification, XLNetTokenizer, AdamW, WarmupLinearSchedule
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from random import Random
torch.set_default_tensor_type('torch.cuda.FloatTensor')


In [0]:
#DATA PREPROCESSING FUNCTIONS

#FUNCTION TO PAD DATA TO UNIFORM LENGTH
def padding(sequences, maxlen=None, dtype=np.int, padding='post', value=0.):
     
    # padding = "pre" pads from the front, "post" pads at the end
    # Function pads to convert lists of strings (of uneven length) into numpy arrays
  
    lengths = [len(s) for s in sequences]

    nb_samples = len(sequences)
    
    if maxlen is None:
        maxlen = np.max(lengths)

    # take the sample shape from the first non empty sequence
    # checking for consistency in the main loop below.
    sample_shape = tuple()
    for s in sequences:
        if len(s) > 0:
            sample_shape = np.asarray(s).shape[1:]
            break

    padded = (np.ones((nb_samples, maxlen) + sample_shape) * value).astype(dtype)
    for idx, s in enumerate(sequences):
        if padding == 'post':
            padded[idx, :len(s)] = s
        elif padding == 'pre':
            padded[idx, -len(s):] = s
        else:
            raise ValueError('Padding type "%s" not understood' % padding)
    return padded
  
#Function to pre-process data for BERT
def process_bert(data,num_train,max_len):
  
  #Import Bert tokenizer
  tokenizer = XLNetTokenizer.from_pretrained('xlnet-base-cased')
  
  #Create arrays to store our tokenised training data
  tokens = []
  tokens_ids = []
  segment_ids = []
  input_masks = []
  
  #Tokenise the training data, add CLS/SEP tokens and then convert to IDs
  for i in range(num_train):

    #Tokenize text and add CLS/SEP tokens, Truncate if over 510 tokens
    text = data[i]["text"]
    token = tokenizer.tokenize(text)
    token_len = len(token)
    if(token_len)>510:
      token = token[-max_len:]
    token.insert(0,'[CLS]')
    token.append('[SEP]')

    #Create segment IDs and inputs masks
    segment_id = [0] * len(token)
    input_mask = [1] * len(token)
    segment_ids.append(segment_id)
    input_masks.append(input_mask)

    #Convert tokens to IDs
    token_id = tokenizer.convert_tokens_to_ids(token)
    tokens_ids.append(token_id)
  

  #Pad the training data so everything is of uniform length  
  tokens_ids = padding(tokens_ids)
  segment_ids = padding(segment_ids)
  input_masks = padding(input_masks)

  max_length = len(tokens_ids[0])


  #Convert training data and labels to tensors
  ids_tensor = torch.tensor([tokens_ids], dtype=torch.long).resize_((num_train,max_length))
  segment_ids_tensor = torch.tensor([segment_ids], dtype=torch.long).resize_((num_train,max_length))
  input_masks_tensor = torch.tensor([input_masks], dtype=torch.long).resize_((num_train,max_length))
  
  return ids_tensor, segment_ids_tensor, input_masks_tensor

#FUNCTION TO CREATE LABELS FOR SMT
def create_SMT_labels(data,num_train):
  labels_tensor = torch.empty(num_train, dtype=torch.long).random_(3)
  labels = data.__getitem__('label')
  labels = labels[0:num_train]

  pos_indices = [i for i, x in enumerate(labels) if x == "positive"]
  neut_indices = [i for i, x in enumerate(labels) if x == "neutral"]
  neg_indices = [i for i, x in enumerate(labels) if x == "negative"]

  labels_tensor[pos_indices] = 0
  labels_tensor[neut_indices] = 1
  labels_tensor[neg_indices] = 2
  
  return labels_tensor

#FUNCTION TO CREATE LABELS FOR IMDB
def create_IMDB_labels(data,num_train):
  labels_tensor = torch.empty(num_train, dtype=torch.long).random_(2)
  labels = data.__getitem__('sentiment')
  labels = labels[0:num_train]

  pos_indices = [i for i, x in enumerate(labels) if x == "pos"]
  neg_indices = [i for i, x in enumerate(labels) if x == "neg"]

  labels_tensor[pos_indices] = 1
  labels_tensor[neg_indices] = 0
  
  return labels_tensor

#FUNCTION TO RESTRICT TRAINING DATASET
def restrict_data_func(train_ids_tensor_, train_segment_ids_tensor_, train_input_masks_tensor_, num_each_label, num_labels, train_labels_tensor_):
  train_ids_tensor = torch.empty((num_each_label*num_labels,max_length), dtype=torch.long).random_(num_labels)
  train_segment_ids_tensor = torch.empty((num_each_label*num_labels,max_length), dtype=torch.long).random_(num_labels)
  train_input_masks_tensor = torch.empty((num_each_label*num_labels,max_length), dtype=torch.long).random_(num_labels)
  train_labels_tensor = torch.empty(num_each_label*num_labels, dtype=torch.long).random_(num_labels)
  for i in range(num_labels):
  
    train_ids_tensor[i*num_each_label:(i+1)*num_each_label] = train_ids_tensor_[train_labels_tensor_==i][:num_each_label]
    train_segment_ids_tensor[i*num_each_label:(i+1)*num_each_label] = train_segment_ids_tensor_[train_labels_tensor_==i][:num_each_label]
    train_input_masks_tensor[i*num_each_label:(i+1)*num_each_label] = train_input_masks_tensor_[train_labels_tensor_==i][:num_each_label] 
 
  for i in range(num_labels):
    train_labels_tensor[i*num_each_label:(i+1)*num_each_label]=train_labels_tensor_[train_labels_tensor_==i][:num_each_label]

  return train_ids_tensor, train_segment_ids_tensor, train_input_masks_tensor, train_labels_tensor

#Function to shuffle a tensor
def shuffle_tensor(tensor):
  torch.manual_seed(25082019)
  print(torch.randperm(tensor.shape[0]))
  tensor = tensor[torch.randperm(tensor.shape[0])]
  print(torch.randperm(tensor.shape[0]))
  print(len(torch.randperm(tensor.shape[0])))
  return tensor

#Import dataset
#train = smt_dataset(train=True)
#valid = smt_dataset(dev=True)
#test = smt_dataset(test=True)

#train = imdb_dataset(train=True)
#test = imdb_dataset(test=True)



In [0]:
!unzip xlnet_imdb_train.zip

In [0]:
# import model for fine-tuning
num_labels = 2
model = XLNetForSequenceClassification.from_pretrained('xlnet-base-cased', num_labels=num_labels)
model.cuda()


In [0]:
#Load preprocessed Data

test_ids_tensor = torch.load('xlnet_imdb_test/test_ids_tensor')
test_segment_ids_tensor = torch.load('xlnet_imdb_test/test_segment_ids_tensor')
test_input_masks_tensor = torch.load('xlnet_imdb_test/test_input_masks_tensor')
test_labels_tensor = torch.load('xlnet_imdb_test/imdb_test_tensor_labels')


train_ids_tensor_ = torch.load('xlnet_imdb_train/train_ids_tensor_')
train_segment_ids_tensor_ = torch.load('xlnet_imdb_train/train_segment_ids_tensor_')
train_input_masks_tensor_ = torch.load('xlnet_imdb_train/train_input_masks_tensor_')
train_labels_tensor_ = torch.load('xlnet_imdb_train/imdb_train_tensor_labels')

In [0]:
#DATA PREP

num_train = 25000
num_test = 25000
max_length = train_ids_tensor_.shape[1]
#torch.manual_seed(25082019)


#Data has already been preprocessed and loaded in via dropbox to save time
#max_len = 510
#train_ids_tensor_, train_segment_ids_tensor_, train_input_masks_tensor_ = process_bert(train,num_train,max_len)
#test_ids_tensor, test_segment_ids_tensor, test_input_masks_tensor = process_bert(test,num_test,max_len)
#train_labels_tensor_ = create_IMDB_labels(train,num_train)
#test_labels_tensor = create_IMDB_labels(test,num_test)


"""
#Set seed so that shuffle is the same for each tensor

#Randomly shuffle training data to get mix of both classes
train_ids_tensor_ = shuffle_tensor(train_ids_tensor_)
train_segment_ids_tensor_ = shuffle_tensor(train_segment_ids_tensor_)
train_input_masks_tensor_ = shuffle_tensor(train_input_masks_tensor_)
train_labels_tensor_ = shuffle_tensor(train_labels_tensor_)


#Split training data into training and validation
num_valid = 2500
valid_ids_tensor = train_ids_tensor_[0:num_valid]
valid_segment_ids_tensor = train_segment_ids_tensor_[0:num_valid]
valid_input_masks_tensor = train_input_masks_tensor_[0:num_valid]
valid_labels_tensor = train_labels_tensor_[0:num_valid]

train_ids_tensor_ = train_ids_tensor_[num_valid:]
train_segment_ids_tensor_ = train_segment_ids_tensor_[num_valid:]
train_input_masks_tensor_= train_input_masks_tensor_[num_valid:]
train_labels_tensor_ = train_labels_tensor_[num_valid:]

"""



#DECIDE HOW MANY OF EACH CLASS TO USE IN TRAINING DATA
restrict_training_data = 1 #Set equal to 0 or 1 
num_each_label=10000

if restrict_training_data:
  train_ids_tensor, train_segment_ids_tensor, train_input_masks_tensor, train_labels_tensor = restrict_data_func(train_ids_tensor_, train_segment_ids_tensor_, train_input_masks_tensor_, num_each_label, num_labels,train_labels_tensor_ )
else:
  train_ids_tensor, train_segment_ids_tensor, train_input_masks_tensor, train_labels_tensor = train_ids_tensor_, train_segment_ids_tensor_, train_input_masks_tensor_, train_labels_tensor_


train_max_batch = 10
valid_max_batch = 10
test_max_batch=10

#Load data into dataloader
train_data = TensorDataset(train_ids_tensor, train_segment_ids_tensor, train_input_masks_tensor, train_labels_tensor)
train_sampler = RandomSampler(train_data)
train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=train_max_batch)

#valid_data = TensorDataset(valid_ids_tensor, valid_segment_ids_tensor, valid_input_masks_tensor, valid_labels_tensor)
#valid_sampler = SequentialSampler(valid_data)
#valid_dataloader = DataLoader(valid_data, sampler=valid_sampler, batch_size=valid_max_batch)


test_data = TensorDataset(test_ids_tensor, test_segment_ids_tensor, test_input_masks_tensor, test_labels_tensor)
test_sampler = SequentialSampler(test_data)
test_dataloader = DataLoader(test_data, sampler=test_sampler, batch_size=test_max_batch)




In [0]:
# Prepare optimizer
param_optimizer = list(model.named_parameters())
no_decay = ['bias', 'LayerNorm.bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
        {'params': [p for n, p in param_optimizer if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
        {'params': [p for n, p in param_optimizer if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
        ]
    
optimizer = AdamW(optimizer_grouped_parameters, lr=3e-5)

In [0]:
#Training
#Set Loss Function
cross_entropy = nn.CrossEntropyLoss()

max_grad_norm = 1.0
# TRAIN loop
model.train()
tr_loss = 0
nb_tr_steps = 0
epochs = 2


for j in range(epochs):
  print("epoch: ", j+1)
  
  print("Begin Training")
  
  for step, batch in enumerate(train_dataloader):
      
      if(step%100==0):
        print("batch: ", step+1)

      # add batch to gpu
      #batch = tuple(t.to(device) for t in batch)
      
      b_train_ids, b_segment_ids, b_input_masks, b_train_labels = batch
      
      
      # forward pass
      outputs = model(input_ids=b_train_ids,token_type_ids=b_segment_ids,attention_mask=b_input_masks,labels=None)
      logits = outputs[0]
      loss = cross_entropy(logits,b_train_labels)
      
      #Calculate accuracy
      
      
      # backward pass
      loss.backward()

      # track train loss
      tr_loss += loss.item()
      nb_tr_steps += 1

      # gradient clipping
      torch.nn.utils.clip_grad_norm_(parameters=model.parameters(), max_norm=max_grad_norm)

      # update parameters
      optimizer.step()
      model.zero_grad()

      
  # print train loss per epoch
  print("Train loss: {}".format(tr_loss/nb_tr_steps))

  
  
print("Begin Validation")

model.eval()
total_correct = 0
for step, batch in enumerate(test_dataloader):

  if(step%100==0):
    print("batch: ", step+1)

  b_valid_ids, b_segment_ids, b_input_masks, b_valid_labels = batch

  with torch.no_grad():
    outputs = model(input_ids=b_valid_ids, token_type_ids=b_segment_ids, attention_mask=b_input_masks)
    logits = outputs[0]


  a = logits.argmax(1)
  b = b_valid_labels

  temp_correct = torch.sum(a==b).cpu().numpy()
  total_correct+=temp_correct
print("Validation Acccuracy:")
print(total_correct/num_test)
    
            
  

In [0]:
"""
model.eval()
total_correct = 0
for step, batch in enumerate(test_dataloader):

  if(step%25==0):
    print("batch: ", step+1)

  b_test_ids, b_segment_ids, b_input_masks, b_valid_labels = batch

  with torch.no_grad():
    logits = model(b_test_ids, token_type_ids=b_segment_ids, attention_mask=b_input_masks)

  a = logits.argmax(1)
  b = b_valid_labels

  temp_correct = torch.sum(a==b).cpu().numpy()
  total_correct+=temp_correct
print("Test Acccuracy:")
print(total_correct/num_test)

"""